In [ ]:
# Copyright 2019 Steven Mattis and Troy Butler

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde as GKDE
from luq.luq import *
import luq.dynamical_systems as ds

The model is the Liénard system, a second order ODE system, which models oscillating circuits:
    $$ u' = v $$
    $$ v' = -u + (\mu - u^2) v. $$
    
The initial conditions are given by $u(0) = u_0 \in \mathbb{R}$ and 
$v(0) = v_0 \in \mathbb{R}$.

The system has a supercritical Hopf bifurcation at $\mu = 0$. There is a 
stable periodic orbit for $\mu > 0$ and the origin is a stable focus for $\mu < 0$. 
See https://www.math.colostate.edu/~shipman/47/volume3b2011/M640_MunozAlicea.pdf 
    for more details.
    
The system is solved numerically using the RK45 method.

A ***true*** distribution of $\mu, u_0$, and $v_0$ are defined by (non-uniform)
Beta distributions and used to generate a set of time series data.

An ***initial*** uniform distribution is assumed and updated by the true time series data.
    
    

In [ ]:
# Uniformly sample the parameter samples to form a "prediction" or "test" set
num_samples = int(1e3)

params = np.random.uniform(size=(num_samples, 1))
param_range = np.array([[-0.5, 0.5]]) # range for nu

ics = np.random.uniform(size=(num_samples, 2))
ic_range = np.array([[0.1, 0.5], [-0.5, -0.1]]) # range for u_0 and v_0  

params = param_range[:, 0] + (param_range[:, 1] - param_range[:, 0]) * params
ics = ic_range[:, 0] + (ic_range[:, 1] - ic_range[:, 0]) * ics

# labels
param_labels = [r'$\mu$']
ic_labels = [r'$u_0$', r'$v_0$']


# Construct the predicted time series data
num_time_preds = int(500)  # number of predictions (uniformly spaced) between [time_start,time_end]
time_start = 0.5
time_end = 40.0
times = np.linspace(time_start, time_end, num_time_preds)

# Solve systems
phys = ds.Lienard()
predicted_time_series = phys.solve(ics=ics, params=params, t_eval=times)

In [ ]:
# Simulate an observed Beta distribution of time series data
num_obs = int(1e3)

true_a = 2
true_b = 2

params_obs = np.random.beta(size=(num_obs, 1), a=true_a, b=true_b)
ics_obs = np.random.beta(size=(num_obs, 2), a=true_a, b=true_b) 

params_obs = param_range[:, 0] + (param_range[:, 1] - param_range[:, 0]) * params_obs
ics_obs = ic_range[:, 0] + (ic_range[:, 1] - ic_range[:, 0]) * ics_obs

# Solve systems
observed_time_series = phys.solve(ics=ics_obs, params=params_obs, t_eval=times)

# Add noise if desired
with_noise = False
noise_stdev = 0.05

if with_noise:
    observed_time_series += noise_stdev * np.random.randn(num_obs, times.shape[0])

In [ ]:
# Use LUQ to learn dynamics and QoIs
learn = LUQ(predicted_data=predicted_time_series, 
            observed_data=observed_time_series)

# time array indices over which to use
time_start_idx = 350
time_end_idx = 499

num_filtered_obs = 50

filtered_times = np.linspace(times[time_start_idx],
                             times[time_end_idx],
                             num_filtered_obs)                          

# Filter data with piecewise linear splines
learn.filter_data(filter_method='splines',
                  predicted_data_coordinates=times,
                  observed_data_coordinates=times,
                  filtered_data_coordinates=filtered_times,
                  tol=3.0e-2, 
                  min_knots=15, 
                  max_knots=40)

In [ ]:
# Learn and classify dynamics.
learn.dynamics(cluster_method='kmeans', kwargs={'n_clusters': 2, 'n_init': 10})

In [ ]:
# Plot clusters of predicted time series
for j in range(learn.num_clusters):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,2.5), gridspec_kw={'width_ratios': [3, 1]}) #(figsize=(10,5))
    ps = []
    for i in range(num_samples):
        if learn.predict_labels[i] == j:
            ps.append(params[i,0])
            ax1.plot(learn.filtered_data_coordinates, learn.filtered_predictions[i, :])
    ax1.set(title='Cluster ' + str(j))
    xs = np.linspace(param_range[0, 0], param_range[0,1], 100)
    ax2.plot(xs, GKDE(ps)(xs))
    ax2.set(xlabel=param_labels[0], title='Param. Distrib.')

In [ ]:
# Plot observed and predicted clusters
for j in range(learn.num_clusters):
    plt.figure()
    cluster_num = j
    for i in range(num_samples):
        if learn.predict_labels[i] == cluster_num:
            plt.plot(learn.filtered_data_coordinates, learn.filtered_predictions[i,:],'b*')
    for i in range(num_obs):
        if learn.obs_labels[i] == cluster_num:
            plt.plot(learn.filtered_data_coordinates, learn.filtered_obs[i,:],'ro')

In [ ]:
# Find best KPCA transformation for given number of QoI and transform time series data.
predict_map, obs_map = learn.learn_qois_and_transform(num_qoi=3)

In [ ]:
# Generate kernel density estimates on new QoI and calculate new weights
pi_predict_kdes = []
pi_obs_kdes = []
r_vals = []
r_means = []
for i in range(learn.num_clusters):
    pi_predict_kdes.append(GKDE(learn.predict_maps[i].T))
    pi_obs_kdes.append(GKDE(learn.obs_maps[i].T))
    r_vals.append(
        np.divide(
            pi_obs_kdes[i](
                learn.predict_maps[i].T), 
            pi_predict_kdes[i](
                learn.predict_maps[i].T)))
    r_means.append(np.mean(r_vals[i]))
print(f'Diagnostics: {r_means}')

In [ ]:
# Compute marginal probablities for each parameter and initial condition.
param_marginals = []
ic_marginals = []
true_param_marginals = []
true_ic_marginals = []
lam_ptr = []
cluster_weights = []
for i in range(learn.num_clusters):
    lam_ptr.append(np.where(learn.predict_labels == i)[0])
    cluster_weights.append(len(np.where(learn.obs_labels == i)[0]) / num_obs)

for i in range(params.shape[1]):
    true_param_marginals.append(GKDE(params_obs[:,i]))
    param_marginals.append([])
    for j in range(learn.num_clusters):
        param_marginals[i].append(GKDE(params[lam_ptr[j], i], weights=r_vals[j]))
        
for i in range(ics.shape[1]):
    true_ic_marginals.append(GKDE(ics_obs[:,i]))
    ic_marginals.append([])
    for j in range(learn.num_clusters):
        ic_marginals[i].append(GKDE(ics[lam_ptr[j], i], weights=r_vals[j]))

In [ ]:
# uniform distribution
def unif_dist(x, p_range):
    y = np.zeros(x.shape)
    val = 1.0/(p_range[1] - p_range[0])
    for i, xi in enumerate(x):
        if xi < p_range[0] or xi >  p_range[1]:
            y[i] = 0
        else:
            y[i] = val
    return y

In [ ]:
# Plot predicted marginal densities for parameters

for i in range(params.shape[1]):
    fig = plt.figure(figsize=(10,10))
    fig.clear()
    x_min = min(min(params[:, i]), min(params_obs[:, i]))
    x_max = max(max(params[:, i]), max(params_obs[:, i]))
    delt = 0.25*(x_max - x_min)
    x = np.linspace(x_min-delt, x_max+delt, 100)
    plt.plot(x, unif_dist(x, param_range[i, :]),
         label = 'Initial guess')
    mar = np.zeros(x.shape)
    for j in range(learn.num_clusters):
        mar += param_marginals[i][j](x) * cluster_weights[j]
    plt.plot(x, mar, label = 'Estimated pullback')
    plt.plot(x, true_param_marginals[i](x), label = 'Actual density')
    plt.title('Comparing pullback to actual density of parameter ' + param_labels[i], fontsize=16)
    plt.legend(fontsize=20)

In [ ]:
# Plot predicted marginal densities for initial conditions.

for i in range(ics.shape[1]):
    fig = plt.figure(figsize=(10,10))
    fig.clear()
    x_min = min(min(ics[:, i]), min(ics_obs[:, i]))
    x_max = max(max(ics[:, i]), max(ics_obs[:, i]))
    delt = 0.25*(x_max - x_min)
    x = np.linspace(x_min-delt, x_max+delt, 100)
    plt.plot(x, unif_dist(x, ic_range[i, :]),
         label = 'Initial guess')
    mar = np.zeros(x.shape)
    for j in range(learn.num_clusters):
        mar += ic_marginals[i][j](x) * cluster_weights[j]
    plt.plot(x, mar, label = 'Estimated pullback')
    plt.plot(x, true_ic_marginals[i](x), label = 'Actual density')
    plt.title('Comparing pullback to actual density of initial condition ' + ic_labels[i], fontsize=16)
    plt.legend(fontsize=20)